In [104]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re
import xml.etree.ElementTree as ET
from tqdm import tqdm

## Load Data

In [105]:
# Load results
hpv_positive_indirect_results = pd.read_csv('Results/HPV Positive indirect results.csv')
hpv_negative_indirect_results = pd.read_csv('Results/HPV Negative indirect results.csv')

### genes
hpv_positive_genes  = pd.read_csv('Results/HPV positive gene results.csv')
hpv_negative_genes = pd.read_csv('Results/HPV negative gene results.csv')

print(f"HPV+ results: {len(hpv_positive_indirect_results)} rows")
print(f"HPV- results: {len(hpv_negative_indirect_results)} rows")

HPV+ results: 99 rows
HPV- results: 64 rows


In [106]:
hpv_positive_indirect_results

,DRUG,GENE_TARGET,CONNECTED_TO (risk gene),Number of risk or immediate neighbor genes targeted,total_genes_targeted_in_drugbank,PERCENTAGE_OF_TARGETS_HIT,ACTION,SPECIFIC_FUNCTION,drug_hypergeom_fdr,drug_empirical_fdr,MUT_TYPE,PMID,NUMBER_OF_ARTICLES,LITERATURE_GENE_TARGETS,RISK_GENE_PMID,RISK_GENE_NUMBER_OF_ARTICLES,RISK_GENE_LITERATURE_GENE_TARGETS
0,1-tert-butyl-3-(4-chloro-phenyl)-1h-pyrazolo[3...,"SRC, LCK, LYN",PIK3CA,3,3,100.000000,UNKNOWN,ATP binding,2.280026e-03,0.020816,SOMATIC,"17961551, 17252232",2,"SRC, LYN","17549376, 15543611, 17848307, 18376308, 1...",17,PIK3CA
1,"2-tert-butyl-9-fluoro-1,6-dihydrobenzo[h]imida...","JAK2, TYK2, JAK3, JAK1",PIK3CA,4,5,80.000000,inhibitor,"acetylcholine receptor binding, ATP binding",4.131506e-04,0.011423,SOMATIC,"15947106, 18204781",2,"JAK2, JAK3","17549376, 15543611, 17848307, 18376308, 1...",17,PIK3CA
2,3-[(3-(2-carboxyethyl)-4-methylpyrrol-2-yl)met...,"FGFR1, FGFR3",PIK3CA,2,2,100.000000,inhibitor,ATP binding,4.122776e-02,0.039860,SOMATIC,"12075207, 16807070, 18059337, 17545628, 116050...",8,"FGFR1, FGFR3","17549376, 15543611, 17848307, 18376308, 1...",17,PIK3CA
3,abrocitinib,"JAK2, TYK2, JAK3, JAK1, JAK2, TYK2, JAK3, JAK1","PIK3CA, PIK3CA",4,4,100.000000,"inhibitor, inhibitor","acetylcholine receptor binding, ATP binding, a...",3.433760e-02,0.038574,"AMPLIFICATION, SOMATIC","15947106, 18204781",4,"JAK2, JAK3","17549376, 15543611, 17848307, 18376308, 1...",34,PIK3CA
4,afatinib,"ERBB4, EGFR, ERBB2",PIK3CA,3,3,100.000000,inhibitor,"ATP binding, actin filament binding",2.280026e-03,0.011423,SOMATIC,"17936670, 12187077, 16540898, 17273794, 157560...",325,"EGFR, ERBB4, ERBB2","17549376, 15543611, 17848307, 18376308, 1...",17,PIK3CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,xl228,"IGF1R, SRC, ABL1",PIK3CA,3,4,75.000000,UNKNOWN,"ATP binding, actin filament binding",8.429235e-03,0.011423,SOMATIC,"17961551, 15221937",2,"IGF1R, SRC","17549376, 15543611, 17848307, 18376308, 1...",17,PIK3CA
95,xl765,"PIK3CD, PIK3CG, PIK3CB, MTOR",PIK3CA,4,5,80.000000,UNKNOWN,"1-phosphatidylinositol-3-kinase activity, ATP ...",5.168885e-06,0.011423,SOMATIC,"15379322, 16927414, 17342092, 15833854, 170470...",8,MTOR,"17549376, 15543611, 17848307, 18376308, 1...",17,PIK3CA
96,xl820,"KDR, PDGFRB, KIT, PDGFRA",PIK3CA,4,4,100.000000,UNKNOWN,ATP binding,9.740999e-05,0.011423,SOMATIC,"16342249, 14499691, 17513510, 16630292, 173178...",9,"PDGFRA, KIT","17549376, 15543611, 17848307, 18376308, 1...",17,PIK3CA
97,xl999,"FGFR1, FGFR3, KDR, PDGFRB, RET, FLT3",PIK3CA,6,6,100.000000,UNKNOWN,ATP binding,1.682879e-07,0.011423,SOMATIC,"12743496, 12670889, 12075207, 15988378, 119386...",55,"FGFR1, RET, FGFR3","17549376, 15543611, 17848307, 18376308, 1...",17,PIK3CA


In [107]:
extracted_target_df= pd.read_csv('Validation pipeline/Results/cleaned_extracted_targets_all_pub_after_2000_GPU_2b_gemma.csv')
extracted_target_df_combined = pd.read_csv('Validation pipeline/Results/cleaned_extracted_combined_targets_all_pub_after_2000_GPU_2b_gemma.csv')

In [108]:
risk_Genes = []
for gene in list(set(hpv_positive_indirect_results['CONNECTED_TO (risk gene)'])):
    temp_risk_gene = gene.split(',')
    temp_risk_gene = [x.strip() for x in temp_risk_gene]
    risk_Genes.extend(temp_risk_gene)
risk_Genes = list(set(risk_Genes))
risk_Genes

### risk genes in the extracted literature

extracted_target_df_filtered = extracted_target_df_combined[extracted_target_df_combined['GENE'].isin(risk_Genes)]
extracted_target_df_filtered['GENE']

1305     CLDN1
4484    PIK3CA
4862      RFC4
5225      SOX2
5656      TLR7
Name: GENE, dtype: object

#### DrugBank

In [109]:
# import xml.etree.ElementTree as ET

# # Load XML
# drugbank_xml = 'Data/DGIDB/drug_bank.xml'
# tree = ET.parse(drugbank_xml)
# root = tree.getroot()

# # Namespace
# ns = {'db': 'http://www.drugbank.ca'}

# Helper to clean tag names
def clean_tag(tag):
    return tag.split('}')[-1] if '}' in tag else tag

# Recursive function to print structure
def print_structure(elem, level=0):
    indent = '  ' * level
    print(f"{indent}- {clean_tag(elem.tag)}")
    for child in elem:
        print_structure(child, level + 1)

# # Get first drug
# first_drug = root.find('db:drug', ns)

# print("🌿 Structure of First Drug Entry:")
# print_structure(first_drug)
# print("\n🌳 Structure of First 3 Drug Entries:")
# drugs = root.findall('db:drug', ns)

# for i, drug in enumerate(drugs[:3]):
#     print(f"\n🔬 Drug {i+1}:")
#     print_structure(drug)


In [110]:
def structure_drug_bank_data(drug_bank_file = 'Data/DGIDB/drug_bank.xml'):
    """
    Function to structure the drug bank data from the XML file.
    :param drug_bank_file: Path to the drug bank XML file.
    :return: DataFrame containing structured drug bank data.
    """
    ### FYI the .find command only finds the first instance of a tag, 
    ### while .findall retrieves all instances of the specified tag within the current element.

    tree = ET.parse(drug_bank_file)
    root = tree.getroot()

    # DrugBank uses a specific namespace
    ns = {'db': 'http://www.drugbank.ca'}
    ### extract all drug elements
    drugs = root.findall('db:drug', ns)
    print(f"Found {len(drugs)} drugs in the DrugBank XML.")
    # Extract drug-gene interactions
    interactions = []
    # The interactions list will store dictionaries with 'drug' and 'gene' keys.
    for drug in root.findall('db:drug', ns): # root.findall('db:drug', ns): Finds all <drug> elements using the namespace.
        drug_name  = drug.find('db:name', ns).text  # drug.find('db:name', ns): Gets the drug's name.
        # print(drug_name)
        for target in drug.findall('db:targets/db:target', ns):  # drug.findall('db:targets/db:target', ns): Finds all <target> elements within <targets>.
            # print(target.tag)
            gene_description = target.find('db:name', ns)  # target.find('db:name', ns): Extracts the gene name for each target.
            poly = target.find('db:polypeptide', ns)  # target.find('db:polypeptide', ns): Extracts the polypeptide information.
            action = target.find('db:actions/db:action', ns) # target.find('db:actions/db:action', ns): Extracts the action of the drug on the target.
            if poly is not None:
                poly_name = poly.find('db:name', ns)
                gene_name = poly.find('db:gene-name', ns)
                specific_function = poly.find('db:specific-function', ns)
                interactions.append({
                    'drug': drug_name,
                    'polypeptide': poly_name.text if poly_name is not None else None,
                    'gene': gene_name.text if gene_name is not None else None,
                    'gene_description': gene_description.text if gene_description is not None else None,
                    'action': action.text if action is not None else None,
                    'specific_function': specific_function.text if specific_function is not None else None
                })
            ############# if polypeptide is not present, we still want to add the drug and gene information
            ############# this is because some drugs may not have a polypeptide associated with them
            ############# but we still want to capture the drug and gene information
            ############# this is common in the DrugBank database, where some drugs target genes directly
            ############# and do not have a polypeptide associated with them

            else:
                gene_name = None
                specific_function = None
                poly_name = None
                action = None
                gene_description = None
                resource = None
                identifier = None
  
                interactions.append({
                        'drug': drug_name,
                        'polypeptide': poly_name.text if poly_name is not None else None,
                        'gene': gene_name.text if gene_name is not None else None,
                        'gene_description': gene_description.text if gene_description is not None else None,
                        'action': action.text if action is not None else None,
                        'specific_function': specific_function.text if specific_function is not None else None
                    })
        
    # Convert to DataFrame
    # Converts the list of dictionaries into a pandas DataFrame, which is easier to analyze, filter, and export.
    df = pd.DataFrame(interactions)

    return df

In [111]:
Drug_bank = structure_drug_bank_data('Data/DGIDB/drug_bank.xml')

Found 17430 drugs in the DrugBank XML.


In [112]:
Drug_bank

,drug,polypeptide,gene,gene_description,action,specific_function
0,Lepirudin,Prothrombin,F2,Prothrombin,inhibitor,calcium ion binding
1,Cetuximab,Epidermal growth factor receptor,EGFR,Epidermal growth factor receptor,binder,actin filament binding
2,Cetuximab,Low affinity immunoglobulin gamma Fc region re...,FCGR3B,Low affinity immunoglobulin gamma Fc region re...,binder,GPI anchor binding
3,Cetuximab,Complement C1q subcomponent subunit A,C1QA,Complement C1q subcomponent subunit A,binder,amyloid-beta binding
4,Cetuximab,Complement C1q subcomponent subunit B,C1QB,Complement C1q subcomponent subunit B,binder,None
...,...,...,...,...,...,...
23131,Benzgalantamine,Muscle nicotinic acetylcholine receptor,None,Muscle nicotinic acetylcholine receptor,allosteric modulator,extracellular ligand-gated monoatomic ion chan...
23132,Zoxazolamine,Small conductance calcium-activated potassium ...,KCNN2,Small conductance calcium-activated potassium ...,activator,alpha-actinin binding
23133,Megestrol,Progesterone receptor,PGR,Progesterone receptor,binder,ATPase binding
23134,AZACYCLONOL,Histamine H1 receptor,HRH1,Histamine H1 receptor,inhibitor,G protein-coupled serotonin receptor activity


#### PPI

In [113]:
protein_interaction = pd.read_csv('Data/Protein-protein interaction data/9606.protein.links.v12.0.txt', sep= ' ')
protein_interaction_full = pd.read_csv('Data/Protein-protein interaction data/9606.protein.links.full.v12.0.txt', sep= ' ')
protein_interaction_detailed = pd.read_csv('Data/Protein-protein interaction data/9606.protein.links.detailed.v12.0.txt', sep= ' ')
### convert proteins to their true names
protein_info = pd.read_csv('Data/Protein-protein interaction data/9606.protein.info.v12.0.txt', on_bad_lines='skip', sep='\t')
protein_aliases= pd.read_csv('Data/Protein-protein interaction data/9606.protein.aliases.v12.0.txt', on_bad_lines='skip', sep='\t')

In [114]:
### convert proteins to their true names
protein_info = pd.read_csv('Data/Protein-protein interaction data/9606.protein.info.v12.0.txt', on_bad_lines='skip', sep='\t')
protein_aliases= pd.read_csv('Data/Protein-protein interaction data/9606.protein.aliases.v12.0.txt', on_bad_lines='skip', sep='\t')

# Method 1: Using the to_dict() method with 'index' as orient
protein_info_translate_name_dict = protein_info.set_index('#string_protein_id')['preferred_name'].to_dict()
protein_alias_translate_name_dict = protein_aliases.set_index('#string_protein_id')['alias'].to_dict()
#print(protein_info_translate_name_dict)

### Protein1
protein1_name = []
for prot_id in tqdm(protein_interaction['protein1']):
    if prot_id in protein_info_translate_name_dict:
        protein1_name.append(protein_info_translate_name_dict[prot_id])
    elif prot_id in protein_alias_translate_name_dict:
        protein1_name.append(protein_alias_translate_name_dict[prot_id])
    else:
        protein1_name.append('')

### Protein 2
protein2_name = []
for prot_id in tqdm(protein_interaction['protein2']):
    if prot_id in protein_info_translate_name_dict:
        protein2_name.append(protein_info_translate_name_dict[prot_id])
    elif prot_id in protein_alias_translate_name_dict:
        protein2_name.append(protein_alias_translate_name_dict[prot_id])
    else:
        protein2_name.append('')

protein_interaction['Translated_protein_1'] = protein1_name
protein_interaction['Translated_protein_2'] = protein2_name

# Create a set of all (protein1, protein2) pairs
ppi_pairs = set(zip(protein_interaction['Translated_protein_1'], protein_interaction['Translated_protein_2']))
# Check for missing reverse pairs
missing_reverse = []
for a, b in ppi_pairs:
    if (b, a) not in ppi_pairs:
        missing_reverse.append((a, b))

print(f"Number of pairs missing their reverse: {len(missing_reverse)}")
if missing_reverse:
    print("Examples:", missing_reverse[:10])
else:
    print("All pairs have their reverse present.")

100%|██████████| 13715404/13715404 [00:21<00:00, 646217.72it/s]



Number of pairs missing their reverse: 0
All pairs have their reverse present.


In [115]:
protein_interaction

,protein1,protein2,combined_score,Translated_protein_1,Translated_protein_2
0,9606.ENSP00000000233,9606.ENSP00000356607,173,ARF5,RALGPS2
1,9606.ENSP00000000233,9606.ENSP00000427567,154,ARF5,FHDC1
2,9606.ENSP00000000233,9606.ENSP00000253413,151,ARF5,ATP6V1E1
3,9606.ENSP00000000233,9606.ENSP00000493357,471,ARF5,CYTH2
4,9606.ENSP00000000233,9606.ENSP00000324127,201,ARF5,PSD3
...,...,...,...,...,...
13715399,9606.ENSP00000501317,9606.ENSP00000475489,195,RFX7,MPHOSPH9
13715400,9606.ENSP00000501317,9606.ENSP00000370447,158,RFX7,VCX
13715401,9606.ENSP00000501317,9606.ENSP00000312272,226,RFX7,YPEL2
13715402,9606.ENSP00000501317,9606.ENSP00000402092,169,RFX7,SAMD3


## Helper Functions

In [116]:
def build_ppi_lookup(protein_interaction_df, confidence_threshold=700):
    """
    Build a set of valid protein-protein interactions from STRING database.
    
    Parameters:
    -----------
    protein_interaction_df : DataFrame
        Protein interaction data with columns: Translated_protein_1, Translated_protein_2, combined_score
    confidence_threshold : int
        Minimum confidence score (default: 700)
    
    Returns:
    --------
    set : Set of (gene1, gene2) tuples for valid interactions (bidirectional)
    """
    # Filter by confidence threshold
    ppi_filtered = protein_interaction_df[
        protein_interaction_df['combined_score'] >= confidence_threshold
    ].copy()
    
    # Remove rows with empty translated proteins
    ppi_filtered = ppi_filtered[
        (ppi_filtered['Translated_protein_1'] != '') & 
        (ppi_filtered['Translated_protein_2'] != '')
    ].copy()
    
    # Create bidirectional set of interactions
    valid_ppi = set()
    for _, row in ppi_filtered.iterrows():
        gene1 = row['Translated_protein_1']
        gene2 = row['Translated_protein_2']
        valid_ppi.add((gene1, gene2))
        valid_ppi.add((gene2, gene1))  # Bidirectional
    
    print(f"Built PPI lookup with {len(valid_ppi)} bidirectional interactions")
    print(f"  (from {len(ppi_filtered)} protein pairs with confidence >= {confidence_threshold})")
    return valid_ppi


def group_gene_families(gene_targets):
    """
    Group related genes into families to reduce visual complexity.
    
    Parameters:
    -----------
    gene_targets : list
        List of gene symbols
    
    Returns:
    --------
    dict : Mapping from original gene to family name (or self if no family)
    """
    family_patterns = {
        'SSTR': r'^SSTR[1-5]$',
        'RPS': r'^RPS\d+[A-Z]*$',
        'RPL': r'^RPL\d+[A-Z]*$',
        'MAPK': r'^MAPK\d+$',
        'MAP2K': r'^MAP2K\d+$',
        'MAP3K': r'^MAP3K\d+$',
        'HLA-A': r'^HLA-A\d+$',
        'HLA-B': r'^HLA-B\d+$',
        'HLA-C': r'^HLA-C\d+$',
        'HIST1H1': r'^HIST1H1[A-Z]$',
        'KRT': r'^KRT\d+[A-Z]*$',
        'FGFR': r'^FGFR[1-4]$',
        'PSM': r'^PSM[A-Z]\d+[A-Z]*$',
        'PSMB': r'^PSMB\d+$',
        'COL': r'^COL\d+A\d+$',
        'CYP': r'^CYP\d+[A-Z]\d+$',
        'ATP': r'^ATP\d+[A-Z]\d*$',
        'SLC': r'^SLC\d+A\d+$',
    }
    
    gene_to_family = {}
    for gene in gene_targets:
        matched = False
        for family, pattern in family_patterns.items():
            if re.match(pattern, gene):
                gene_to_family[gene] = f"{family} family"
                matched = True
                break
        if not matched:
            gene_to_family[gene] = gene  # Keep original if no family match
    
    return gene_to_family

## Core Processing Function

In [117]:
def prepare_sankey_data(results_df, ppi_set, top_n=50, 
                        required_drugs=None, article_threshold=0):
    """
    Prepare filtered and verified link data for Sankey diagrams.
    
    Parameters:
    -----------
    results_df : DataFrame
        Drug repurposing results with columns:
        - DRUG, LITERATURE_GENE_TARGETS, RISK_GENE_LITERATURE_GENE_TARGETS
        - drug_empirical_fdr, PERCENTAGE_OF_TARGETS_HIT, NUMBER_OF_ARTICLES
    ppi_set : set
        Valid protein-protein interactions from build_ppi_lookup()
    top_n : int
        Number of top drugs to include
    required_drugs : list, optional
        Drug names that must be included (case-insensitive)
    article_threshold : int
        Minimum NUMBER_OF_ARTICLES (default: 0)
    
    Returns:
    --------
    dict with keys:
        - 'link_counts': DataFrame of aggregated links
        - 'drugs': list of drug names
        - 'gene_targets': list of gene target names (literature-validated)
        - 'risk_genes': list of risk gene names (literature-validated)
        - 'top_df': Top N filtered results
    """
    # Filter by article threshold
    results_filtered = results_df[results_df['NUMBER_OF_ARTICLES'] > article_threshold].copy()
    
    # Sort by FDR (ascending) and target percentage (descending)
    results_sorted = results_filtered.sort_values(
        by=['drug_empirical_fdr', 'PERCENTAGE_OF_TARGETS_HIT'],
        ascending=[True, False]
    )
    
    # Handle required drugs
    if required_drugs:
        # Normalize required drugs to lowercase for comparison
        required_drugs_lower = [drug.lower() for drug in required_drugs]
        
        # Separate required drugs from the rest
        required_mask = results_sorted['DRUG'].str.lower().isin(required_drugs_lower)
        required_df = results_sorted[required_mask].copy()
        remaining_df = results_sorted[~required_mask].copy()
        
        # Take required drugs + fill remaining slots with top-ranked drugs
        n_required = len(required_df)
        n_remaining = max(0, top_n - n_required)
        
        top_df = pd.concat([required_df, remaining_df.head(n_remaining)], ignore_index=True)
        
        print(f"   Including {n_required} required drugs + {n_remaining} top-ranked drugs")
    else:
        # Get top N by ranking only
        top_df = results_sorted.head(top_n).copy()
    
    # Build expanded links with automatic expansion to reach top_n valid drugs
    expanded_links = []
    drugs_with_connections = []
    drugs_checked = 0
    
    # If we have required drugs, prioritize checking them first
    if required_drugs:
        required_drugs_lower = [drug.lower() for drug in required_drugs]
        
        # First, process required drugs
        for _, row in results_sorted.iterrows():
            drug = row['DRUG']
            
            # Skip if not a required drug
            if drug.lower() not in required_drugs_lower:
                continue
                
            drugs_checked += 1
            
            # Parse literature-validated gene targets only
            gene_targets_str = str(row.get('LITERATURE_GENE_TARGETS', ''))
            gene_targets = [g.strip() for g in gene_targets_str.split(',') if g.strip() and g.strip() != 'nan']
            
            # Parse literature-validated risk genes only
            risk_genes_str = str(row.get('RISK_GENE_LITERATURE_GENE_TARGETS', ''))
            risk_genes = [g.strip() for g in risk_genes_str.split(',') if g.strip() and g.strip() != 'nan']
            
            # Check if this drug has any valid connections
            drug_has_connections = False
            drug_links = []
            
            for gene_target in gene_targets:
                # Check if this gene target has any PPI-verified connections to risk genes
                has_connection = False
                target_to_risk_links = []
                
                for risk_gene in risk_genes:
                    if (gene_target, risk_gene) in ppi_set:
                        has_connection = True
                        target_to_risk_links.append({
                            'source': gene_target,
                            'target': risk_gene,
                            'link_type': 'target_to_risk'
                        })
                
                # Only add drug -> gene_target link if gene_target connects to at least one risk gene
                if has_connection:
                    drug_has_connections = True
                    drug_links.append({
                        'source': drug,
                        'target': gene_target,
                        'link_type': 'drug_to_target'
                    })
                    drug_links.extend(target_to_risk_links)
            
            # If this required drug has valid connections, add it
            if drug_has_connections:
                drugs_with_connections.append(drug)
                expanded_links.extend(drug_links)
    
    # Now fill remaining slots with top-ranked drugs (excluding already added drugs)
    for _, row in results_sorted.iterrows():
        # Stop if we already have enough drugs
        if len(drugs_with_connections) >= top_n:
            break
            
        drug = row['DRUG']
        
        # Skip if already added
        if drug in drugs_with_connections:
            continue
        
        drugs_checked += 1
        
        # Parse literature-validated gene targets only
        gene_targets_str = str(row.get('LITERATURE_GENE_TARGETS', ''))
        gene_targets = [g.strip() for g in gene_targets_str.split(',') if g.strip() and g.strip() != 'nan']
        
        # Parse literature-validated risk genes only
        risk_genes_str = str(row.get('RISK_GENE_LITERATURE_GENE_TARGETS', ''))
        risk_genes = [g.strip() for g in risk_genes_str.split(',') if g.strip() and g.strip() != 'nan']
        
        # Check if this drug has any valid connections
        drug_has_connections = False
        drug_links = []
        
        for gene_target in gene_targets:
            # Check if this gene target has any PPI-verified connections to risk genes
            has_connection = False
            target_to_risk_links = []
            
            for risk_gene in risk_genes:
                if (gene_target, risk_gene) in ppi_set:
                    has_connection = True
                    target_to_risk_links.append({
                        'source': gene_target,
                        'target': risk_gene,
                        'link_type': 'target_to_risk'
                    })
            
            # Only add drug -> gene_target link if gene_target connects to at least one risk gene
            if has_connection:
                drug_has_connections = True
                drug_links.append({
                    'source': drug,
                    'target': gene_target,
                    'link_type': 'drug_to_target'
                })
                drug_links.extend(target_to_risk_links)
        
        # If this drug has valid connections, add it
        if drug_has_connections:
            drugs_with_connections.append(drug)
            expanded_links.extend(drug_links)
    
    print(f"   Checked {drugs_checked} drugs to find {len(drugs_with_connections)} with valid connections")
    
    # Convert to DataFrame and get unique links (remove duplicates)
    links_df = pd.DataFrame(expanded_links)
    link_counts = links_df.drop_duplicates(subset=['source', 'target', 'link_type']).copy()
    # Set all values to 1 (uniform thickness - only show flow, not weight)
    link_counts['value'] = 1
    
    # For drugs with too many gene targets, keep only top gene targets by connection count
    max_targets_per_drug = 15  # Threshold for filtering
    
    # Count connections per gene target for each drug
    drug_to_target_links = link_counts[link_counts['link_type'] == 'drug_to_target'].copy()
    target_to_risk_links = link_counts[link_counts['link_type'] == 'target_to_risk'].copy()
    
    for drug in drug_to_target_links['source'].unique():
        # Get gene targets for this drug
        drug_targets = drug_to_target_links[drug_to_target_links['source'] == drug]['target'].unique()
        
        # If this drug has too many targets, filter to top targets by connection count
        if len(drug_targets) > max_targets_per_drug:
            # Count outgoing connections (to risk genes) for each gene target
            target_connection_counts = {}
            for target in drug_targets:
                n_connections = len(target_to_risk_links[target_to_risk_links['source'] == target])
                target_connection_counts[target] = n_connections
            
            # Sort targets by connection count (descending) and keep top max_targets_per_drug
            top_targets = sorted(target_connection_counts.items(), key=lambda x: x[1], reverse=True)[:max_targets_per_drug]
            top_target_names = [t[0] for t in top_targets]
            
            # Remove drug->target links for targets not in top list
            link_counts = link_counts[
                ~((link_counts['link_type'] == 'drug_to_target') & 
                  (link_counts['source'] == drug) & 
                  (~link_counts['target'].isin(top_target_names)))
            ].copy()
            
            # DON'T remove target->risk links globally - they might be used by other drugs!
            # The individual Sankey function will only show targets that are connected to the drug
            
            print(f"   Filtered {drug}: {len(drug_targets)} → {len(top_target_names)} gene targets (kept top by connection count)")
    
    # Remove genes that appear in both layers (prevent loops)
    # Only remove if a gene is BOTH a target AND a risk gene for the SAME drug path
    gene_targets_all = link_counts[link_counts['link_type'] == 'drug_to_target']['target'].unique()
    risk_genes_all = link_counts[link_counts['link_type'] == 'target_to_risk']['target'].unique()
    overlap = set(gene_targets_all) & set(risk_genes_all)
    
    if overlap:
        print(f"   Warning: {len(overlap)} genes appear in both layers: {', '.join(sorted(list(overlap)))}")
        # Instead of removing all overlapping genes, only remove drug→target links for overlapping genes
        # Keep target→risk links since those are the actual pathways we want to show
        links_before = len(link_counts)
        link_counts = link_counts[
            ~((link_counts['link_type'] == 'drug_to_target') & (link_counts['target'].isin(overlap)))
        ].copy()
        links_after = len(link_counts)
        print(f"   Removed {links_before - links_after} drug→target links for overlapping genes to prevent loops")
    
    # Apply gene family grouping
    gene_targets_cleaned = link_counts[link_counts['link_type'] == 'drug_to_target']['target'].unique()
    gene_family_map = group_gene_families(gene_targets_cleaned)
    
    # Create reverse mapping: family name -> list of individual genes
    ### for the legend
    family_to_genes = {}
    for gene, family in gene_family_map.items():
        if family != gene:  # Only track families (not individual genes)
            if family not in family_to_genes:
                family_to_genes[family] = []
            family_to_genes[family].append(gene)
    
    # Sort genes within each family
    for family in family_to_genes:
        family_to_genes[family] = sorted(family_to_genes[family])
    
    # Update gene targets to families
    link_counts['target'] = link_counts.apply(
        lambda row: gene_family_map.get(row['target'], row['target']) 
        if row['link_type'] == 'drug_to_target' else row['target'], axis=1
    )
    
    link_counts['source'] = link_counts.apply(
        lambda row: gene_family_map.get(row['source'], row['source']) 
        if row['link_type'] == 'target_to_risk' else row['source'], axis=1
    )

    # Re-aggregate after grouping (remove duplicates created by family grouping)
    link_counts = link_counts.drop_duplicates(subset=['source', 'target', 'link_type']).copy()
    # Reset all values to 1 (uniform thickness)
    link_counts['value'] = 1    
    
    # Extract final node lists
    drugs = link_counts[link_counts['link_type'] == 'drug_to_target']['source'].unique().tolist()
    gene_targets = link_counts[link_counts['link_type'] == 'drug_to_target']['target'].unique().tolist()
    risk_genes = link_counts[link_counts['link_type'] == 'target_to_risk']['target'].unique().tolist()
    
    # Update top_df to only include drugs that made it through filtering
    top_df = results_sorted[results_sorted['DRUG'].isin(drugs_with_connections)].copy()
    
    print(f"\n📊 Prepared data summary:")
    print(f"   Links: {len(link_counts)}")
    print(f"   Drugs: {len(drugs)}")
    print(f"   Gene Targets: {len(gene_targets)}")
    print(f"   Risk Genes: {len(risk_genes)}")
    
    return {
        'link_counts': link_counts,
        'drugs': drugs,
        'gene_targets': gene_targets,
        'risk_genes': risk_genes,
        'top_df': top_df,
        'family_to_genes': family_to_genes }   


## Visualization Functions

In [118]:
def create_combined_sankey(data_dict, title="Drug Repurposing Pathways", 
                          height=1400, width=2400):
    """
    Create a combined Sankey diagram showing all drugs.
    
    Parameters:
    -----------
    data_dict : dict
        Output from prepare_sankey_data()
    title : str
        Diagram title
    height, width : int
        Figure dimensions
    
    Returns:
    --------
    plotly.graph_objects.Figure
    """
    link_counts = data_dict['link_counts']
    drugs = set(data_dict['drugs'])
    gene_targets = set(data_dict['gene_targets'])
    
    # Filter link_counts to only include gene targets that are actually connected to drugs
    # Get gene targets that have drug→target connections
    connected_targets = link_counts[link_counts['link_type'] == 'drug_to_target']['target'].unique()
    
    # Filter out target→risk links for targets that aren't connected to any drug
    link_counts = link_counts[
        ~((link_counts['link_type'] == 'target_to_risk') & 
          (~link_counts['source'].isin(connected_targets)))
    ].copy()
    
    # Identify gene families present
    gene_families = sorted([gt for gt in gene_targets if 'family' in gt])
    
    # Build node list and mapping
    all_nodes = sorted(list(set(link_counts['source'].unique()) | set(link_counts['target'].unique())))
    node_to_idx = {node: idx for idx, node in enumerate(all_nodes)}
    
    # Create link indices
    sources = [node_to_idx[row['source']] for _, row in link_counts.iterrows()]
    targets = [node_to_idx[row['target']] for _, row in link_counts.iterrows()]
    values = link_counts['value'].tolist()
    
    # Assign colors
    node_colors = []
    node_x = []
    for node in all_nodes:
        if node in drugs:
            node_colors.append('rgba(31, 119, 180, 0.8)')  # Blue
            node_x.append(0.001)
        elif node in gene_targets:
            node_colors.append('rgba(44, 160, 44, 0.8)')   # Green
            node_x.append(0.5)
        else:
            node_colors.append('rgba(214, 39, 40, 0.8)')   # Red
            node_x.append(0.999)
    
    # Create figure
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=5,
            line=dict(color="black", width=0.5),
            label=all_nodes,
            color=node_colors,
            x=node_x
        ),
        link=dict(
            source=sources,
            target=targets,
            value=values,
            color='rgba(0, 0, 0, 0.2)'
        )
    )])
    
    fig.update_layout(
        title={'text': title, 'x': 0.5, 'xanchor': 'center'},
        font=dict(size=12),
        height=height,
        width=width
    )
    
    # Add gene family legend if families are present
    if gene_families:
        family_to_genes = data_dict.get('family_to_genes', {})
        legend_lines = ["<b>Gene Families:</b>"]
        
        for family in gene_families:
            if family in family_to_genes:
                genes = family_to_genes[family]
                # Show first 5 genes, indicate if there are more
                if len(genes) <= 5:
                    gene_list = ", ".join(genes)
                else:
                    gene_list = ", ".join(genes[:5]) + f", ... (+{len(genes)-5})"
                legend_lines.append(f"• <b>{family}</b>: {gene_list}")
            else:
                legend_lines.append(f"• {family}")
        
        legend_text = "<br>".join(legend_lines)
        fig.add_annotation(
            text=legend_text,
            xref="paper", yref="paper",
            x=0.01, y=1.05,
            xanchor="left", yanchor="top",
            showarrow=False,
            font=dict(size=10, family="Arial"),
            align="left",
            bgcolor="rgba(255, 255, 255, 0.9)",
            bordercolor="black",
            borderwidth=1,
            borderpad=8
        )
    
    return fig


def create_individual_sankeys(data_dict, drug_list=None, title_prefix="",
                              ncols=2, height=2200, width=2000, 
                              risk_gene_threshold=0):
    """
    Create a grid of individual Sankey diagrams, one per drug.
    
    Parameters:
    -----------
    data_dict : dict
        Output from prepare_sankey_data()
    drug_list : list, optional
        Specific drugs to include. If None, uses top drugs from data_dict
    title_prefix : str
        Prefix for overall title (e.g., "HPV+ " or "HPV- ")
    ncols : int
        Number of columns in grid
    height, width : int
        Figure dimensions
    risk_gene_threshold : int
        Minimum connections to include a risk gene (reduces clutter)
        Set to 0 to include all.
    
    Returns:
    --------
    plotly.graph_objects.Figure
    """
    link_counts = data_dict['link_counts']
    gene_targets = set(data_dict['gene_targets'])
    
    # Get all gene families in the data
    all_gene_families = sorted([gt for gt in gene_targets if 'family' in gt])
    
    if drug_list is None:
        drug_list = data_dict['drugs'][:10]  # Default to top 10
    
    # Calculate grid dimensions
    n_drugs = len(drug_list)
    nrows = (n_drugs + ncols - 1) // ncols  # Ceiling division
    
    # Create subplots
    subplot_titles = [drug.title() for drug in drug_list]
    fig = make_subplots(
        rows=nrows,
        cols=ncols,
        subplot_titles=subplot_titles,
        specs=[[{'type': 'sankey'}] * ncols for _ in range(nrows)],
        vertical_spacing=0.05,
        horizontal_spacing=0.03
    )
    
    # Process each drug
    for idx, drug in enumerate(drug_list):
        row = (idx // ncols) + 1
        col = (idx % ncols) + 1
        
        # Get links for this drug
        # First, get drug→target links for this specific drug
        drug_to_target_links = link_counts[
            (link_counts['link_type'] == 'drug_to_target') & 
            (link_counts['source'] == drug)
        ]
        
        # Get the gene targets for this drug
        drug_gene_targets = drug_to_target_links['target'].unique()
        
        # Get target→risk links where the source is one of this drug's gene targets
        target_to_risk_links = link_counts[
            (link_counts['link_type'] == 'target_to_risk') &
            (link_counts['source'].isin(drug_gene_targets))
        ]
        
        # Combine both link types
        drug_links = pd.concat([drug_to_target_links, target_to_risk_links])
        
        if len(drug_links) == 0:
            print(f"   ⚠️  Skipping {drug}: No links found after getting drug_to_target and target_to_risk")
            continue
        
        # Debug: Check if we have target→risk links
        drug_to_target_count = len(drug_links[drug_links['link_type'] == 'drug_to_target'])
        target_to_risk_count = len(drug_links[drug_links['link_type'] == 'target_to_risk'])
        
        if target_to_risk_count == 0 and drug_to_target_count > 0:
            print(f"   ⚠️  Skipping {drug}: Has {drug_to_target_count} gene targets but NO target→risk connections")
            continue
        
        # Only apply filtering if this drug has too many gene targets (threshold: 10)
        max_targets_threshold = 15
        
        if len(drug_gene_targets) > max_targets_threshold:
            # Filter risk genes by connection threshold for drugs with many targets
            target_to_risk = drug_links[drug_links['link_type'] == 'target_to_risk']
            risk_gene_counts = target_to_risk.groupby('target').size()
            risk_genes_keep = risk_gene_counts[risk_gene_counts >= risk_gene_threshold].index.tolist()
            
            drug_links = drug_links[
                (drug_links['link_type'] == 'drug_to_target') |
                ((drug_links['link_type'] == 'target_to_risk') & 
                 (drug_links['target'].isin(risk_genes_keep)))
            ].copy()
            
            if len(drug_links) == 0:
                continue
            
            # Remove gene targets that have no downstream connections after risk gene filtering
            drug_to_target_filtered = drug_links[drug_links['link_type'] == 'drug_to_target']
            target_to_risk_filtered = drug_links[drug_links['link_type'] == 'target_to_risk']
            
            # Get gene targets that still have at least one connection to a risk gene
            gene_targets_with_connections = target_to_risk_filtered['source'].unique()
            
            # Keep only drug→target links where the target has downstream connections
            drug_links = drug_links[
                (drug_links['link_type'] == 'target_to_risk') |
                ((drug_links['link_type'] == 'drug_to_target') & 
                 (drug_links['target'].isin(gene_targets_with_connections)))
            ].copy()
            
            if len(drug_links) == 0:
                continue
        
        # Build node list in correct order: drug first, then gene targets, then risk genes
        # This ensures proper left-to-right flow in the Sankey diagram
        drug_to_target_final = drug_links[drug_links['link_type'] == 'drug_to_target']
        target_to_risk_final = drug_links[drug_links['link_type'] == 'target_to_risk']
        
        # Get unique nodes in each layer
        drug_nodes_layer = [drug]  # Always just the one drug
        gene_target_nodes_layer = sorted(drug_to_target_final['target'].unique().tolist())
        risk_gene_nodes_layer = sorted(target_to_risk_final['target'].unique().tolist())
        
        # Combine in correct order
        drug_nodes = drug_nodes_layer + gene_target_nodes_layer + risk_gene_nodes_layer
        node_to_idx = {node: i for i, node in enumerate(drug_nodes)}
        
        # Create link indices with constant values for uniform thickness
        sources = [node_to_idx[row['source']] for _, row in drug_links.iterrows()]
        targets = [node_to_idx[row['target']] for _, row in drug_links.iterrows()]
        # Use constant value of 1 for all links - thickness controlled by arrangement parameter
        values = [1] * len(drug_links)
        
        # Assign colors
        node_colors = []
        for node in drug_nodes:
            if node == drug:
                node_colors.append('rgba(31, 119, 180, 0.9)')  # Blue
            elif node in gene_targets:
                node_colors.append('rgba(44, 160, 44, 0.9)')   # Green
            else:
                node_colors.append('rgba(214, 39, 40, 0.9)')   # Red
        
        # Add trace
        fig.add_trace(
            go.Sankey(
                arrangement='snap',
                valueformat='.0f',
                node=dict(
                    pad=8,
                    thickness=5,
                    line=dict(color="black", width=0.5),
                    label=drug_nodes,
                    color=node_colors
                ),
                link=dict(
                    source=sources,
                    target=targets,
                    value=values,
                    color='rgba(0, 0, 0, 0.2)'
                ),
                textfont=dict(size=15, color='black', family='Arial')
            ),
            row=row,
            col=col
        )
    
    # Update layout
    fig.update_layout(
        title={
            'text': f"{title_prefix}Individual Drug Pathways<br>",
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 22}
        },
        height=height,
        width=width,
        showlegend=False,
        font=dict(size=14),
        margin=dict(t=100, b=20, l=20, r=20)
    ) 
    
    # Update subplot titles (drug names)
    for annotation in fig['layout']['annotations']:
        annotation['font'] = dict(size=15, color='black', family='Arial')
    
    # Add gene family legend if families are present
    if all_gene_families:
        family_to_genes = data_dict.get('family_to_genes', {})
        legend_lines = ["<b>Gene Families:</b>"]
        
        for family in all_gene_families:
            if family in family_to_genes:
                genes = family_to_genes[family]
                # Show first 3 genes for individual panels (less space)
                if len(genes) <= 3:
                    gene_list = ", ".join(genes)
                else:
                    gene_list = ", ".join(genes[:3]) + f", ... (+{len(genes)-3})"
                legend_lines.append(f"• <b>{family}</b>: {gene_list}")
            else:
                legend_lines.append(f"• {family}")
        
        legend_text = "<br>".join(legend_lines)
        fig.add_annotation(
            text=legend_text,
            xref="paper", yref="paper",
            x=0.01, y=1,
            xanchor="left", yanchor="top",
            showarrow=False,
            font=dict(size=9, family="Arial"),
            align="left",
            bgcolor="rgba(255, 255, 255, 0.9)",
            bordercolor="black",
            borderwidth=1,
            borderpad=6
        )
    
    return fig

## Wrapper Functions

In [119]:
### Initialize PPI lookup
# Build PPI lookup (only needs to be done once)
ppi_set = build_ppi_lookup(protein_interaction, confidence_threshold=700)

Built PPI lookup with 473860 bidirectional interactions
  (from 473860 protein pairs with confidence >= 700)


In [120]:
def create_sankey_diagrams(cohort='HPV+', top_n=10, required_drugs=None,
                          show_combined=True, show_individual=True, ppi_set = ppi_set,
                          article_threshold=0):
    """
    One-function interface to create all Sankey diagrams for a cohort.
    
    Parameters:
    -----------
    cohort : str
        'HPV+' or 'HPV-'
    top_n : int
        Number of top drugs to include
    required_drugs : list, optional
        Drug names that must be included (case-insensitive)
    show_combined : bool
        Whether to show combined multi-drug diagram
    show_individual : bool
        Whether to show individual per-drug diagrams
    article_threshold : int
        Minimum NUMBER_OF_ARTICLES (default: 0)
    
    Returns:
    --------
    dict with keys 'combined_fig' and 'individual_fig' (if requested)
    """
    # Select data
    if cohort.upper() == 'HPV+':
        results_df = hpv_positive_indirect_results
        title_prefix = "HPV+ "
    else:
        results_df = hpv_negative_indirect_results
        title_prefix = "HPV- "
    
    # Prepare data
    print(f"\n{'='*60}")
    print(f"Processing {cohort} cohort (Top {top_n})")
    print(f"{'='*60}")
    
    data_dict = prepare_sankey_data(
        results_df=results_df,
        ppi_set= ppi_set,
        top_n=top_n,
        required_drugs=required_drugs,
        article_threshold=article_threshold
    )
    
    results = {}
    results['data'] = data_dict  # Save the data for debugging
    
    # Combined diagram
    if show_combined:
        print(f"\n🎨 Creating combined diagram...")
        results['combined_fig'] = create_combined_sankey(
            data_dict,
            title=f"{title_prefix}Top {top_n} Medications: Drug → Gene Target → Risk Gene"
        )
        results['combined_fig'].show()
    
    # Individual diagrams
    if show_individual:
        print(f"\n🎨 Creating individual drug diagrams...")
        
        # Build drug list with required drugs first
        drug_list = []
        if required_drugs:
            for req_drug in required_drugs:
                for drug in data_dict['drugs']:
                    if drug.lower() == req_drug.lower() and drug not in drug_list:
                        drug_list.append(drug)
                        break
        
        # Add remaining drugs to reach top_n
        for drug in data_dict['drugs']:
            if drug not in drug_list:
                drug_list.append(drug)
            if len(drug_list) == top_n:
                break
        
        results['individual_fig'] = create_individual_sankeys(
            data_dict,
            drug_list=drug_list,
            title_prefix=title_prefix
        )
        results['individual_fig'].show()
    
    return results

### Create Filtered Top 50 Tables (Only PPI-Validated Connections)

In [121]:
Drug_bank

,drug,polypeptide,gene,gene_description,action,specific_function
0,Lepirudin,Prothrombin,F2,Prothrombin,inhibitor,calcium ion binding
1,Cetuximab,Epidermal growth factor receptor,EGFR,Epidermal growth factor receptor,binder,actin filament binding
2,Cetuximab,Low affinity immunoglobulin gamma Fc region re...,FCGR3B,Low affinity immunoglobulin gamma Fc region re...,binder,GPI anchor binding
3,Cetuximab,Complement C1q subcomponent subunit A,C1QA,Complement C1q subcomponent subunit A,binder,amyloid-beta binding
4,Cetuximab,Complement C1q subcomponent subunit B,C1QB,Complement C1q subcomponent subunit B,binder,None
...,...,...,...,...,...,...
23131,Benzgalantamine,Muscle nicotinic acetylcholine receptor,None,Muscle nicotinic acetylcholine receptor,allosteric modulator,extracellular ligand-gated monoatomic ion chan...
23132,Zoxazolamine,Small conductance calcium-activated potassium ...,KCNN2,Small conductance calcium-activated potassium ...,activator,alpha-actinin binding
23133,Megestrol,Progesterone receptor,PGR,Progesterone receptor,binder,ATPase binding
23134,AZACYCLONOL,Histamine H1 receptor,HRH1,Histamine H1 receptor,inhibitor,G protein-coupled serotonin receptor activity


In [122]:
def create_filtered_top_table(results_df, ppi_set, drug_bank_df, top_n=50, cohort_name=''):
    """
    Create a filtered table showing only drugs with PPI-validated connections.
    Mimics the filtering used in Sankey diagrams.
    
    Parameters:
    -----------
    results_df : DataFrame
        Drug repurposing results
    ppi_set : set
        Valid protein-protein interactions
    drug_bank_df : DataFrame
        DrugBank data with drug-gene-action mappings
    top_n : int
        Number of top drugs to include
    cohort_name : str
        Name for display (e.g., 'HPV+' or 'HPV-')
    
    Returns:
    --------
    DataFrame with filtered and formatted results
    """
    # Load HPV gene results for mutation type mapping
    if cohort_name.upper() == 'HPV+':
        hpv_gene_results = pd.read_csv('Results/HPV positive gene results.csv')
    else:
        hpv_gene_results = pd.read_csv('Results/HPV negative gene results.csv')
    
    # Create gene -> MUT_TYPE mapping
    gene_mut_type_map = {}
    for _, gene_row in hpv_gene_results.iterrows():
        gene = gene_row['gene_name']
        mut_type = gene_row['MUT_TYPE']
        gene_mut_type_map[gene] = mut_type
    
    # Sort by FDR and target percentage
    results_sorted = results_df.sort_values(
        by=['drug_empirical_fdr', 'PERCENTAGE_OF_TARGETS_HIT'],
        ascending=[True, False]
    ).copy()
    
    # Process each drug to filter connections
    filtered_data = []
    
    for idx, row in results_sorted.iterrows():
        drug = row['DRUG']
        
        # Parse literature-validated gene targets
        lit_targets_str = str(row.get('LITERATURE_GENE_TARGETS', ''))
        lit_targets = [g.strip() for g in lit_targets_str.split(',') 
                      if g.strip() and g.strip() != 'nan']
        
        # Parse literature-validated risk genes
        lit_risk_str = str(row.get('RISK_GENE_LITERATURE_GENE_TARGETS', ''))
        lit_risk = [g.strip() for g in lit_risk_str.split(',') 
                   if g.strip() and g.strip() != 'nan']
        
        # Find PPI-validated connections
        validated_targets = set()
        validated_risks = set()
        
        if lit_targets and lit_risk: ### double check both lists exist
            for gene_target in lit_targets:
                for risk_gene in lit_risk:
                    if (gene_target, risk_gene) in ppi_set:
                        validated_targets.add(gene_target)
                        validated_risks.add(risk_gene)
        
        # Only include drugs with valid connections
        if validated_targets and validated_risks:
            # Get article counts - these are already integers in the DataFrame
            target_article_count = row.get('NUMBER_OF_ARTICLES', 0)
            risk_article_count = row.get('RISK_GENE_NUMBER_OF_ARTICLES', 0)
            
            # Convert to int if they're not already
            if pd.notna(target_article_count):
                target_article_count = int(target_article_count)
            else:
                target_article_count = 0
                
            if pd.notna(risk_article_count):
                risk_article_count = int(risk_article_count)
            else:
                risk_article_count = 0
            
            # Get PMIDs - clean up whitespace
            target_pmids_raw = str(row.get('PMID', ''))
            risk_pmids_raw = str(row.get('RISK_GENE_PMID', ''))
            
            # Clean up PMIDs (remove 'nan', extra spaces)
            target_pmids = target_pmids_raw if target_pmids_raw != 'nan' else ''
            risk_pmids = risk_pmids_raw if risk_pmids_raw != 'nan' else ''
            
            # Clean up extra spaces in PMID strings
            if target_pmids:
                target_pmids = ', '.join([p.strip() for p in target_pmids.split(',') if p.strip()])
            if risk_pmids:
                risk_pmids = ', '.join([p.strip() for p in risk_pmids.split(',') if p.strip()])
            
            # Verify ACTION from DrugBank for validated gene targets only
            verified_actions = []
            verified_functions = []
            
            # Filter DrugBank for this drug (case-insensitive)
            drug_entries = drug_bank_df[drug_bank_df['drug'].str.lower() == drug.lower()]
            
            # For each validated target, check if it's in DrugBank
            for target in sorted(validated_targets):
                action_found = False
                if len(drug_entries) > 0:
                    target_entries = drug_entries[drug_entries['gene'] == target]
                    
                    if len(target_entries) > 0:
                        # Get actions for this target
                        for _, target_row in target_entries.iterrows():
                            action = target_row.get('action', None)
                            function = target_row.get('specific_function', None)
                            
                            if action and pd.notna(action) and action != 'None':
                                verified_actions.append(f"{target}: {action}")
                                action_found = True
                            
                            if function and pd.notna(function) and function != 'None':
                                if function not in verified_functions:
                                    verified_functions.append(function)
                
                # If no action found for this target, mark as UNKNOWN
                if not action_found:
                    verified_actions.append(f"{target}: UNKNOWN")
            
            # Create action and function strings
            action_str = '; '.join(verified_actions)
            
            if verified_functions:
                function_str = '; '.join(verified_functions[:3])  # Limit to first 3 functions
            else:
                function_str = row.get('SPECIFIC_FUNCTION', 'UNKNOWN')
            
            # Get mutation types for validated risk genes only
            mut_types = set()
            for risk_gene in sorted(validated_risks):
                if risk_gene in gene_mut_type_map:
                    # Split by comma in case a gene has multiple mutation types
                    gene_mut_types = gene_mut_type_map[risk_gene]
                    if isinstance(gene_mut_types, str):
                        # Split and clean each mutation type
                        for mt in gene_mut_types.split(','):
                            mut_types.add(mt.strip())
            
            # Create MUT_TYPE string (join multiple types if present)
            if mut_types:
                mut_type_str = ', '.join(sorted(mut_types))
            else:
                mut_type_str = row.get('MUT_TYPE', '')  # Fallback to original if not found
            
            # Add all drugs (including those with UNKNOWN actions)
            filtered_data.append({
                'DRUG': drug,
                'MUT_TYPE': mut_type_str,
                'Literature-Validated Gene Targets': ', '.join(sorted(validated_targets)),
                'Number of Validated Targets': len(validated_targets),
                'Literature-Validated Risk Genes': ', '.join(sorted(validated_risks)),
                'Number of Validated Risk Genes': len(validated_risks),
                'Gene Target Article Count': target_article_count,
                'Risk Gene Article Count': risk_article_count,
                'Gene Target PMIDs': target_pmids,
                'Risk Gene PMIDs': risk_pmids,
                'ACTION': action_str,
                'SPECIFIC_FUNCTION': function_str,
                'Drug Empirical FDR': row.get('drug_empirical_fdr', ''),
                'Percent Targets Hit': row.get('PERCENTAGE_OF_TARGETS_HIT', '')
            })
        
        # Stop when we have enough drugs
        if len(filtered_data) >= top_n:
            break
    
    # Create DataFrame
    result_df = pd.DataFrame(filtered_data)
    
    if len(result_df) > 0:
        print(f"\n{'='*80}")
        print(f"{cohort_name} Top {top_n} Drugs with PPI-Validated Connections")
        print(f"{'='*80}")
        print(f"Total drugs with valid connections: {len(result_df)}")
        print(f"{'='*80}\n")
    
    return result_df

#### HPV+

In [132]:
# Create HPV+ Top 50 Table
hpv_pos_top50 = create_filtered_top_table(
    hpv_positive_indirect_results, 
    ppi_set,
    Drug_bank,
    top_n=50, 
    cohort_name='HPV+'
)

# Display table
hpv_pos_top50


HPV+ Top 50 Drugs with PPI-Validated Connections
Total drugs with valid connections: 50



,DRUG,MUT_TYPE,Literature-Validated Gene Targets,Number of Validated Targets,Literature-Validated Risk Genes,Number of Validated Risk Genes,Gene Target Article Count,Risk Gene Article Count,Gene Target PMIDs,Risk Gene PMIDs,ACTION,SPECIFIC_FUNCTION,Drug Empirical FDR,Percent Targets Hit
0,quercetin,"AMPLIFICATION, SOMATIC","ESR1, HCK, STAT3",3,PIK3CA,1,44,17,"17225522, 11222718, 16503733, 11773428, 162241...","17549376, 15543611, 17848307, 18376308, 163809...",ESR1: UNKNOWN; HCK: UNKNOWN; STAT3: inhibitor,14-3-3 protein binding; ATP binding; chromatin...,0.007679,46.875000
1,foreskin fibroblast (neonatal),DELETION,TNF,1,TLR7,1,6,1,"12075207, 16888677, 11926139, 12569015, 125484...",17201162,TNF: agonist,cytokine activity,0.008516,72.727273
2,afatinib,"AMPLIFICATION, SOMATIC","EGFR, ERBB2, ERBB4",3,PIK3CA,1,325,17,"17936670, 12187077, 16540898, 17273794, 157560...","17549376, 15543611, 17848307, 18376308, 163809...",EGFR: inhibitor; ERBB2: inhibitor; ERBB4: inhi...,actin filament binding; ATP binding,0.011423,100.000000
3,amuvatinib,"AMPLIFICATION, SOMATIC","KIT, MET, PDGFRA, RET",4,PIK3CA,1,154,34,"12743496, 12670889, 15988378, 15735049, 153197...","17549376, 15543611, 17848307, 18376308, 163809...",KIT: modulator; MET: UNKNOWN; PDGFRA: modulato...,ATP binding,0.011423,100.000000
4,bms-754807,"AMPLIFICATION, SOMATIC","AKT1, IGF1R",2,PIK3CA,1,7,17,"16002527, 14581353, 17974918, 15896313, 152219...","17549376, 15543611, 17848307, 18376308, 163809...",AKT1: inhibitor; IGF1R: inhibitor,14-3-3 protein binding; ATP binding,0.011423,100.000000
5,brigatinib,"AMPLIFICATION, SOMATIC","ALK, EGFR, ERBB2, ERBB4, IGF1R, MET",6,PIK3CA,1,725,34,"17936670, 15319714, 12187077, 16540898, 172737...","17549376, 15543611, 17848307, 18376308, 163809...",ALK: inhibitor; EGFR: inhibitor; ERBB2: inhibi...,ATP binding; actin filament binding,0.011423,100.000000
6,cabozantinib,"AMPLIFICATION, SOMATIC","MET, RET",2,PIK3CA,1,58,17,"12743496, 12670889, 15988378, 15735049, 153197...","17549376, 15543611, 17848307, 18376308, 163809...",MET: antagonist; RET: antagonist,ATP binding,0.011423,100.000000
7,capivasertib,"AMPLIFICATION, SOMATIC","AKT1, AKT2",2,PIK3CA,1,8,17,"14581353, 16002527, 17974918, 15896313, 167780...","17549376, 15543611, 17848307, 18376308, 163809...",AKT1: inhibitor; AKT2: inhibitor,14-3-3 protein binding; ATP binding,0.011423,100.000000
8,dacomitinib,"AMPLIFICATION, SOMATIC","EGFR, ERBB2, ERBB4",3,PIK3CA,1,325,17,"17936670, 12187077, 16540898, 17273794, 157560...","17549376, 15543611, 17848307, 18376308, 163809...",EGFR: inhibitor; ERBB2: antagonist; ERBB4: ant...,actin filament binding; ATP binding,0.011423,100.000000
9,debio-1347,"AMPLIFICATION, SOMATIC","FGFR1, FGFR2, FGFR3",3,PIK3CA,1,10,17,"12075207, 16807070, 18059337, 17545628, 116050...","17549376, 15543611, 17848307, 18376308, 163809...",FGFR1: inhibitor; FGFR2: inhibitor; FGFR3: inh...,ATP binding,0.011423,100.000000


#### HPV-

In [ ]:
# Create HPV- Top 50 Table
hpv_neg_top50 = create_filtered_top_table(
    hpv_negative_indirect_results, 
    ppi_set,
    Drug_bank,
    top_n=50, 
    cohort_name='HPV-'
)
# Display table
hpv_neg_top50


HPV- Top 50 Drugs with PPI-Validated Connections
Total drugs with valid connections: 50



,DRUG,MUT_TYPE,Literature-Validated Gene Targets,Number of Validated Targets,Literature-Validated Risk Genes,Number of Validated Risk Genes,Gene Target Article Count,Risk Gene Article Count,Gene Target PMIDs,Risk Gene PMIDs,ACTION,SPECIFIC_FUNCTION,Drug Empirical FDR,Percent Targets Hit
0,lucitanib,"AMPLIFICATION, SOMATIC","FGFR1, FGFR2, FGFR3, PDGFRA",4,PIK3CA,1,12,17,"12075207, 16807070, 14499691, 18059337, 175456...","14581353, 18447972, 16380997, 16807070, 179903...",FGFR1: inhibitor; FGFR2: inhibitor; FGFR3: inh...,ATP binding,0.004461,100.000000
1,enzastaurin,"AMPLIFICATION, DELETION, SOMATIC","AKT1, AURKA, CHEK1",3,"CDKN2A, PIK3CA",2,14,33,"16002527, 14581353, 17974918, 15896313, 168886...","14586645, 17990317, 17549376, 18376308, 117204...",AKT1: inhibitor; AURKA: UNKNOWN; CHEK1: UNKNOWN,14-3-3 protein binding; ATP binding,0.004461,88.888889
2,semaxanib,"AMPLIFICATION, SOMATIC","FGFR1, KIT, PDGFRA, RET",4,PIK3CA,1,61,17,"12743496, 12670889, 15988378, 11938683, 174356...","14581353, 18447972, 16380997, 16807070, 179903...",FGFR1: inhibitor; KIT: inhibitor; PDGFRA: inhi...,ATP binding,0.004461,88.888889
3,quercetin,"AMPLIFICATION, SOMATIC","ESR1, HCK, STAT3",3,PIK3CA,1,44,17,"17225522, 11222718, 16503733, 11773428, 162241...","14581353, 18447972, 16380997, 16807070, 179903...",ESR1: UNKNOWN; HCK: UNKNOWN; STAT3: inhibitor,14-3-3 protein binding; ATP binding; chromatin...,0.004461,59.375000
4,fn-1501,"DELETION, SOMATIC","CDK2, CDK4, CDK6",3,"CDKN2A, CDKN2B",2,33,18,"16952836, 16027075, 12017338, 11564579, 158338...","14586645, 16713672, 16849682, 14654786, 150831...",CDK2: inhibitor; CDK4: inhibitor; CDK6: inhibitor,ATP binding,0.005511,100.000000
5,hymenialdisine,"DELETION, SOMATIC","CDK1, CDK2, CDK5",3,"CDKN2A, CDKN2B",2,16,18,"15543230, 11585773, 11453659, 11272092, 112679...","14586645, 16713672, 16849682, 14654786, 150831...",CDK1: UNKNOWN; CDK2: inhibitor; CDK5: inhibitor,ATP binding; acetylcholine receptor activator ...,0.005511,100.000000
6,olomoucine,"DELETION, SOMATIC","CDK1, CDK2, CDK5",3,"CDKN2A, CDKN2B",2,16,18,"15543230, 11585773, 11453659, 11272092, 112679...","14586645, 16713672, 16849682, 14654786, 150831...",CDK1: binder; CDK2: inhibitor; CDK5: inhibitor,ATP binding; acetylcholine receptor activator ...,0.005511,100.000000
7,su9516,"DELETION, SOMATIC","CDK1, CDK2, CDK5",3,"CDKN2A, CDKN2B",2,16,18,"15543230, 11585773, 11453659, 11272092, 112679...","14586645, 16713672, 16849682, 14654786, 150831...",CDK1: binder; CDK2: inhibitor; CDK5: inhibitor,ATP binding; acetylcholine receptor activator ...,0.005511,100.000000
8,purvalanol,"DELETION, SOMATIC","CDK1, CDK2, CDK4, CDK5",4,"CDKN2A, CDKN2B",2,30,18,"11272092, 16027075, 12017338, 11564579, 158338...","14586645, 16713672, 16849682, 14654786, 150831...",CDK1: binder; CDK2: inhibitor; CDK4: UNKNOWN; ...,ATP binding; acetylcholine receptor activator ...,0.005511,55.555556
9,midostaurin,"AMPLIFICATION, SOMATIC","KIT, PDGFRA",2,PIK3CA,1,9,17,"16342249, 14499691, 17513510, 16630292, 173178...","14581353, 18447972, 16380997, 16807070, 179903...",KIT: antagonist; PDGFRA: antagonist,ATP binding,0.008146,100.000000


### Export Tables to CSV

In [125]:
# Export to CSV
hpv_pos_top50.to_csv('Results/HPV_Positive_Top50_PPI_Validated.csv', index=False)
hpv_neg_top50.to_csv('Results/HPV_Negative_Top50_PPI_Validated.csv', index=False)

print("✅ Tables exported successfully!")
print(f"   - HPV+ Top 50: {len(hpv_pos_top50)} drugs")
print(f"   - HPV- Top 50: {len(hpv_neg_top50)} drugs")

✅ Tables exported successfully!
   - HPV+ Top 50: 50 drugs
   - HPV- Top 50: 50 drugs


### HPV+ Top 10 with Required Drugs

In [126]:
# HPV+ cohort: Top 10 drugs, ensure golomotid and fostamatinib are included
results_hpv_pos = create_sankey_diagrams(
    cohort='HPV+',
    top_n=10,
    show_combined=True,
    show_individual=True,
    article_threshold=0
)


Processing HPV+ cohort (Top 10)
   Checked 11 drugs to find 10 with valid connections

📊 Prepared data summary:
   Links: 43
   Drugs: 10
   Gene Targets: 16
   Risk Genes: 2

🎨 Creating combined diagram...



🎨 Creating individual drug diagrams...


### HPV- Top 10 with Required Drugs

In [127]:
# HPV- cohort: Top 10 drugs, ensure artenimol and fostamatinib are included
results_hpv_neg = create_sankey_diagrams(
    cohort='HPV-',
    top_n=10,
    required_drugs=['artenimol', 'fostamatinib'],
    show_combined=True,
    show_individual=True,
    article_threshold=0
)


Processing HPV- cohort (Top 10)
   Including 2 required drugs + 8 top-ranked drugs
   Checked 10 drugs to find 10 with valid connections
   Filtered fostamatinib: 36 → 15 gene targets (kept top by connection count)

📊 Prepared data summary:
   Links: 144
   Drugs: 10
   Gene Targets: 27
   Risk Genes: 16

🎨 Creating combined diagram...



🎨 Creating individual drug diagrams...


### Create Individual Diagram for Specific Drug

### Single drug double checks

In [128]:
def debug_drug_filtering(drug_name, cohort='HPV+'):
    """
    Debug why a drug might not appear in the final Sankey diagram.
    
    Parameters:
    -----------
    drug_name : str
        Name of the drug to investigate
    cohort : str
        'HPV+' or 'HPV-'
    """
    # Select data
    if cohort.upper() == 'HPV+':
        results_df = hpv_positive_indirect_results
    else:
        results_df = hpv_negative_indirect_results
    
    # Find the drug
    drug_row = results_df[results_df['DRUG'].str.lower() == drug_name.lower()]
    
    if len(drug_row) == 0:
        print(f"❌ Drug '{drug_name}' not found in {cohort} data")
        return
    
    drug_row = drug_row.iloc[0]
    
    print(f"\n{'='*70}")
    print(f"🔍 Debugging: {drug_name.upper()} in {cohort}")
    print(f"{'='*70}")
    
    # Check basic info
    print(f"\n1️⃣  BASIC INFO:")
    print(f"   FDR: {drug_row.get('drug_empirical_fdr', 'N/A')}")
    print(f"   Target %: {drug_row.get('PERCENTAGE_OF_TARGETS_HIT', 'N/A')}")
    print(f"   Articles: {drug_row.get('NUMBER_OF_ARTICLES', 'N/A')}")
    
    # Check literature-validated gene targets
    lit_targets_str = str(drug_row.get('LITERATURE_GENE_TARGETS', ''))
    lit_targets = [g.strip() for g in lit_targets_str.split(',') if g.strip() and g.strip() != 'nan']
    
    print(f"\n2️⃣  LITERATURE-VALIDATED GENE TARGETS:")
    print(f"   Count: {len(lit_targets)}")
    if lit_targets:
        print(f"   Targets: {', '.join(lit_targets[:10])}")
        if len(lit_targets) > 10:
            print(f"   ... and {len(lit_targets) - 10} more")
    else:
        print(f"   ⚠️  NO LITERATURE-VALIDATED GENE TARGETS")
    
    # Check literature-validated risk genes
    lit_risk_str = str(drug_row.get('RISK_GENE_LITERATURE_GENE_TARGETS', ''))
    lit_risk = [g.strip() for g in lit_risk_str.split(',') if g.strip() and g.strip() != 'nan']
    
    print(f"\n3️⃣  LITERATURE-VALIDATED RISK GENES:")
    print(f"   Count: {len(lit_risk)}")
    if lit_risk:
        print(f"   Risk genes: {', '.join(lit_risk[:10])}")
        if len(lit_risk) > 10:
            print(f"   ... and {len(lit_risk) - 10} more")
    else:
        print(f"   ⚠️  NO LITERATURE-VALIDATED RISK GENES")
    
    # Check PPI connections
    if lit_targets and lit_risk:
        print(f"\n4️⃣  PPI-VERIFIED CONNECTIONS:")
        connections_found = []
        for gene_target in lit_targets:
            for risk_gene in lit_risk:
                if (gene_target, risk_gene) in ppi_set:
                    connections_found.append((gene_target, risk_gene))
        
        print(f"   Valid connections: {len(connections_found)}")
        if connections_found:
            print(f"   Examples:")
            for gt, rg in connections_found[:5]:
                print(f"      • {gt} → {rg}")
            if len(connections_found) > 5:
                print(f"      ... and {len(connections_found) - 5} more")
        else:
            print(f"   ❌ NO PPI-VERIFIED CONNECTIONS FOUND")
            print(f"   This is why the drug doesn't appear in the diagram!")
    
    # Check ranking
    results_sorted = results_df.sort_values(
        by=['drug_empirical_fdr', 'PERCENTAGE_OF_TARGETS_HIT'],
        ascending=[True, False]
    )
    rank = results_sorted[results_sorted['DRUG'].str.lower() == drug_name.lower()].index[0]
    actual_rank = (results_sorted.index == rank).argmax() + 1
    
    print(f"\n5️⃣  RANKING:")
    print(f"   Position in sorted list: #{actual_rank}")
    print(f"   Top 10 threshold: {'✅ YES' if actual_rank <= 10 else '❌ NO'}")
    
    print(f"\n{'='*70}\n")

# Example usage:
debug_drug_filtering('semaxanib', cohort='HPV-')


🔍 Debugging: SEMAXANIB in HPV-

1️⃣  BASIC INFO:
   FDR: 0.0044609077718746
   Target %: 88.88888888888889
   Articles: 61

2️⃣  LITERATURE-VALIDATED GENE TARGETS:
   Count: 4
   Targets: FGFR1, PDGFRA, KIT, RET

3️⃣  LITERATURE-VALIDATED RISK GENES:
   Count: 1
   Risk genes: PIK3CA

4️⃣  PPI-VERIFIED CONNECTIONS:
   Valid connections: 4
   Examples:
      • FGFR1 → PIK3CA
      • PDGFRA → PIK3CA
      • KIT → PIK3CA
      • RET → PIK3CA

5️⃣  RANKING:
   Position in sorted list: #3
   Top 10 threshold: ✅ YES




In [129]:
def check_top_n_drugs(cohort='HPV+', top_n=10):
    """
    Check which of the top N drugs have valid connections and which don't.
    
    Parameters:
    -----------
    cohort : str
        'HPV+' or 'HPV-'
    top_n : int
        Number of top drugs to check
    """
    # Select data
    if cohort.upper() == 'HPV+':
        results_df = hpv_positive_indirect_results
    else:
        results_df = hpv_negative_indirect_results
    
    # Sort drugs
    results_sorted = results_df.sort_values(
        by=['drug_empirical_fdr', 'PERCENTAGE_OF_TARGETS_HIT'],
        ascending=[True, False]
    )
    
    top_drugs = results_sorted.head(top_n)
    
    print(f"\n{'='*70}")
    print(f"🔍 Checking Top {top_n} Drugs in {cohort}")
    print(f"{'='*70}\n")
    
    drugs_with_connections = []
    drugs_without_connections = []
    
    for idx, (_, row) in enumerate(top_drugs.iterrows(), 1):
        drug = row['DRUG']
        
        # Parse literature-validated gene targets
        lit_targets_str = str(row.get('LITERATURE_GENE_TARGETS', ''))
        lit_targets = [g.strip() for g in lit_targets_str.split(',') if g.strip() and g.strip() != 'nan']
        
        # Parse literature-validated risk genes
        lit_risk_str = str(row.get('RISK_GENE_LITERATURE_GENE_TARGETS', ''))
        lit_risk = [g.strip() for g in lit_risk_str.split(',') if g.strip() and g.strip() != 'nan']
        
        # Check PPI connections
        connections_found = 0
        if lit_targets and lit_risk:
            for gene_target in lit_targets:
                for risk_gene in lit_risk:
                    if (gene_target, risk_gene) in ppi_set:
                        connections_found += 1
        
        if connections_found > 0:
            drugs_with_connections.append(drug)
            print(f"✅ #{idx}. {drug}")
            print(f"    → {len(lit_targets)} lit targets, {len(lit_risk)} lit risk genes")
            print(f"    → {connections_found} PPI-verified connections\n")
        else:
            drugs_without_connections.append(drug)
            print(f"❌ #{idx}. {drug}")
            print(f"    → {len(lit_targets)} lit targets, {len(lit_risk)} lit risk genes")
            print(f"    → NO PPI-verified connections (will not appear in diagram)\n")
    
    print(f"{'='*70}")
    print(f"SUMMARY:")
    print(f"  Drugs WITH valid connections: {len(drugs_with_connections)}/{top_n}")
    print(f"  Drugs WITHOUT connections: {len(drugs_without_connections)}/{top_n}")
    
    if drugs_without_connections:
        print(f"\n  Skipped drugs: {', '.join(drugs_without_connections)}")
    
    print(f"{'='*70}\n")

# Check HPV+ top 10
check_top_n_drugs('HPV-', top_n=10)


🔍 Checking Top 10 Drugs in HPV-

✅ #1. lucitanib
    → 4 lit targets, 1 lit risk genes
    → 4 PPI-verified connections

✅ #2. enzastaurin
    → 3 lit targets, 2 lit risk genes
    → 4 PPI-verified connections

✅ #3. semaxanib
    → 4 lit targets, 1 lit risk genes
    → 4 PPI-verified connections

✅ #4. quercetin
    → 4 lit targets, 1 lit risk genes
    → 3 PPI-verified connections

✅ #5. fn-1501
    → 3 lit targets, 2 lit risk genes
    → 6 PPI-verified connections

✅ #6. hymenialdisine
    → 3 lit targets, 2 lit risk genes
    → 4 PPI-verified connections

✅ #7. olomoucine
    → 3 lit targets, 2 lit risk genes
    → 4 PPI-verified connections

✅ #8. su9516
    → 3 lit targets, 2 lit risk genes
    → 4 PPI-verified connections

✅ #9. purvalanol
    → 4 lit targets, 2 lit risk genes
    → 6 PPI-verified connections

✅ #10. midostaurin
    → 2 lit targets, 1 lit risk genes
    → 2 PPI-verified connections

SUMMARY:
  Drugs WITH valid connections: 10/10
  Drugs WITHOUT connections: 0/1

In [130]:
# Create diagram for a single specific drug (must be in top 50)
def create_single_drug_diagram(drug_name, cohort='HPV+'):
    """
    Create Sankey diagram for a single drug.
    
    Parameters:
    -----------
    drug_name : str
        Name of the drug (case-insensitive)
    cohort : str
        'HPV+' or 'HPV-'
    """
    # Select data
    if cohort.upper() == 'HPV+':
        results_df = hpv_positive_indirect_results
    else:
        results_df = hpv_negative_indirect_results
    
    # Prepare data with top 50
    data_dict = prepare_sankey_data(
        results_df=results_df,
        ppi_set=ppi_set,
        top_n=100,
        article_threshold=0
    )
    
    # Check if drug exists
    drug_match = None
    for drug in data_dict['drugs']:
        if drug.lower() == drug_name.lower():
            drug_match = drug
            break
    
    if drug_match is None:
        print(f"❌ Drug '{drug_name}' not found in top 50 for {cohort}")
        print(f"Available drugs: {', '.join(data_dict['drugs'][:20])}...")
        return None
    
    # Create individual diagram
    fig = create_individual_sankeys(
        data_dict,
        drug_list=[drug_match],
        title_prefix=f"{cohort} ",
        ncols=1,
        height=800,
        width=1200
    )
    
    fig.show()
    return fig

# Example: Create diagram for artenimol in HPV-
# create_single_drug_diagram('artenimol', cohort='HPV-')

In [131]:
### examples of checking specific genes and drugs
create_single_drug_diagram('fostamatinib', cohort='HPV-')

   Checked 64 drugs to find 62 with valid connections
   Filtered fostamatinib: 36 → 15 gene targets (kept top by connection count)
   Removed 8 drug→target links for overlapping genes to prevent loops

📊 Prepared data summary:
   Links: 357
   Drugs: 59
   Gene Targets: 54
   Risk Genes: 19
